In [110]:
import pandas as pd
import numpy as np

def home_loan_calculator(home_price, leverage, var_r, freq=12, total_term=30, 
                         fix_term=0, hybrid_fix=0, fix_r=0, offset_ini=0, offset_reg=0, 
                         offset_only_for_compare=False, compare_inst_in_fix=0,
                         compare_inst_after_fix=0, compare_offset_interest=0):
    
    fix_inst = -np.pmt(fix_r/freq, total_term*freq, home_price*(1-leverage)*hybrid_fix,0)
    var_inst = -np.pmt(var_r/freq, total_term*freq, home_price*(1-leverage)*(1-hybrid_fix),0)

    payment_dict = {}
    fix_bal0 = hybrid_fix * (1-leverage) * home_price
    var_bal0 = (1-hybrid_fix) * (1-leverage) * home_price
    offset0 = offset_ini
    fix_term_end_flag = 1
    counter = 1

    for t_yr in range(1, total_term+1, 1):
        temp_dict = {'year': t_yr}

        for t_prd in range(1, freq+1, 1):

            temp_dict["term #"] = t_prd
            temp_dict["beginning balance"] = fix_bal0 + var_bal0

            if t_yr > fix_term and fix_term_end_flag:
                var_bal0 += fix_bal0
                fix_bal0 = 0
                var_inst = -np.pmt(var_r/freq, (total_term-fix_term)*freq, var_bal0,0)
                fix_inst = 0
                fix_term_end_flag = 0


            fix_int = fix_bal0 * fix_r/freq
            fix_pcp = fix_inst - fix_int
            fix_bal1 = fix_bal0 - fix_pcp

            temp_dict["fix: beginning balance"] = fix_bal0
            temp_dict["fix: interest"] = fix_int
            temp_dict["fix: principal"] = fix_pcp
            temp_dict["fix: ending balance"] = fix_bal1
            

            if offset_only_for_compare:
                var_int = var_bal0 * var_r/freq
            else:
                var_int = (var_bal0 - offset0) * var_r/freq
                
            var_pcp = var_inst - var_int
            var_bal1 = var_bal0 - var_pcp

            temp_dict["var: beginning balance"] = var_bal0
            temp_dict["var: interest"] = var_int
            temp_dict["var: principal"] = var_pcp
            temp_dict["var: ending balance"] = var_bal1
            temp_dict["var: offset balance"] = offset0        

            temp_dict["interest"] = fix_int + var_int
            temp_dict["installment"] = fix_inst + var_inst
            temp_dict["ending balance"] = fix_bal1 + var_bal1
            temp_dict["ending balance after offset"] = fix_bal1 + var_bal1 - offset0        

            fix_bal0 = fix_bal1
            var_bal0 = var_bal1
            
            if compare_offset_interest > 0:
                offset0 *= (1 + compare_offset_interest/freq)
            
            offset0 += offset_reg
            
            if compare_inst_in_fix > 0 and fix_term_end_flag:
                offset0 += var_inst + fix_inst - compare_inst_in_fix
                
            if compare_inst_after_fix > 0 and not fix_term_end_flag:
                offset0 += var_inst + fix_inst - compare_inst_after_fix

            if temp_dict["ending balance after offset"] < 0:
                break

            payment_dict[counter] = temp_dict.copy()
            counter += 1
            
    return pd.DataFrame.from_dict(payment_dict).transpose()


In [118]:
cba_loan = {
    "home_price": 657121,
    "leverage": 0.3,
    "total_term": 30,
    "fix_term": 4,
    "hybrid_fix": 0.85,
    "fix_r": 0.0189,
    "var_r": 0.026,
    "offset_ini": 3000,
    "offset_reg": 230,
    "freq": 26 # freq: monthly = 12, fornightly = 26, weekly = 52
}

ubank_loan = {
    "home_price": 657121,
    "leverage": 0.3,
    "total_term": 30,
    "fix_term": 3,
    "hybrid_fix": 1,
    "fix_r": 0.0175,
    "var_r": 0.0234,
    "offset_ini": 3000,
    "offset_reg": 230*26/12,
    "freq": 12, # freq: monthly = 12, fornightly = 26, weekly = 52
    "offset_only_for_compare": True,
    "compare_inst_in_fix": 785*26/12,
    "compare_inst_after_fix": 839*26/12,
    "compare_offset_interest": 0.0126
}

In [112]:
home_loan_calculator(**ubank_loan)

,year,term #,beginning balance,fix: beginning balance,fix: interest,fix: principal,fix: ending balance,var: beginning balance,var: interest,var: principal,var: ending balance,var: offset balance,interest,installment,ending balance,ending balance after offset
1,1.0,1.0,459984.700000,459984.700000,670.811021,972.454179,459012.245821,0.000000,0.000000,0.000000,0.000000,3000.000000,670.811021,1643.26520,459012.245821,456012.245821
2,1.0,2.0,459012.245821,459012.245821,669.392858,973.872341,458038.373480,0.000000,0.000000,0.000000,0.000000,3443.915200,669.392858,1643.26520,458038.373480,454594.458280
3,1.0,3.0,458038.373480,458038.373480,667.972628,975.292572,457063.080908,0.000000,0.000000,0.000000,0.000000,3888.296511,667.972628,1643.26520,457063.080908,453174.784397
4,1.0,4.0,457063.080908,457063.080908,666.550326,976.714874,456086.366034,0.000000,0.000000,0.000000,0.000000,4333.144422,666.550326,1643.26520,456086.366034,451753.221612
5,1.0,5.0,456086.366034,456086.366034,665.125950,978.139249,455108.226785,0.000000,0.000000,0.000000,0.000000,4778.459423,665.125950,1643.26520,455108.226785,450329.767361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,23.0,3.0,151612.584025,0.000000,0.000000,0.000000,0.000000,151612.584025,295.644539,1471.158681,150141.425344,140838.175494,295.644539,1766.80322,150141.425344,9303.249850
268,23.0,4.0,150141.425344,0.000000,0.000000,0.000000,0.000000,150141.425344,292.775779,1474.027440,148667.397904,141433.358798,292.775779,1766.80322,148667.397904,7234.039105
269,23.0,5.0,148667.397904,0.000000,0.000000,0.000000,0.000000,148667.397904,289.901426,1476.901794,147190.496110,142029.167045,289.901426,1766.80322,147190.496110,5161.329065
270,23.0,6.0,147190.496110,0.000000,0.000000,0.000000,0.000000,147190.496110,287.021467,1479.781752,145710.714358,142625.600890,287.021467,1766.80322,145710.714358,3085.113468


In [114]:
home_loan_calculator(**cba_loan)

,year,term #,beginning balance,fix: beginning balance,fix: interest,fix: principal,fix: ending balance,var: beginning balance,var: interest,var: principal,var: ending balance,var: offset balance,interest,installment,ending balance,ending balance after offset
1,1.0,1.0,459984.700000,390986.995000,615.804517,807.947303,390179.047697,68997.705000,142.995027,133.230186,68864.474814,3000.0,758.799545,1699.977033,459043.522511,456043.522511
2,1.0,2.0,459043.522511,390179.047697,614.532000,809.219820,389369.827877,68864.474814,141.623029,134.602184,68729.872630,3500.0,756.155029,1699.977033,458099.700507,454599.700507
3,1.0,3.0,458099.700507,389369.827877,613.257479,810.494341,388559.333536,68729.872630,140.248057,135.977156,68593.895474,4000.0,753.505536,1699.977033,457153.229010,453153.229010
4,1.0,4.0,457153.229010,388559.333536,611.980950,811.770870,387747.562666,68593.895474,138.870107,137.355106,68456.540368,4500.0,750.851057,1699.977033,456204.103033,451704.103033
5,1.0,5.0,456204.103033,387747.562666,610.702411,813.049409,386934.513257,68456.540368,137.489171,138.736042,68317.804325,5000.0,748.191582,1699.977033,455252.317582,450252.317582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,21.0,10.0,139106.304885,0.000000,0.000000,0.000000,0.000000,139106.304885,25.146994,1791.824250,137314.480635,127500.0,25.146994,1816.971244,137314.480635,9814.480635
251,21.0,11.0,137314.480635,0.000000,0.000000,0.000000,0.000000,137314.480635,20.181375,1796.789869,135517.690766,128000.0,20.181375,1816.971244,135517.690766,7517.690766
252,21.0,12.0,135517.690766,0.000000,0.000000,0.000000,0.000000,135517.690766,15.204997,1801.766247,133715.924519,128500.0,15.204997,1816.971244,133715.924519,5215.924519
253,22.0,1.0,133715.924519,0.000000,0.000000,0.000000,0.000000,133715.924519,10.217836,1806.753407,131909.171111,129000.0,10.217836,1816.971244,131909.171111,2909.171111


In [115]:
home_loan_calculator(**cba_loan)["interest"].sum()

126007.69358381745

In [119]:
home_loan_calculator(**cba_loan)["interest"].sum()

125870.364571078